In [6]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import ttest_ind
from itertools import combinations
import numpy as np

# Generate sample data
np.random.seed(42)
num_records = 1000
commuting_zones = ['CZ{}'.format(i) for i in range(1, 51)]
states = ['State{}'.format(i) for i in range(1, 11)]
cohorts = [1980, 1981, 1982, 1983, 1984]
ages = list(range(20, 81))
years = list(range(1996, 2013))

table1a_data = {
    'par_cz': np.random.choice(commuting_zones, num_records),
    'par_stateabbrv': np.random.choice(states, num_records),
    'cohort': np.random.choice(cohorts, num_records),
    'inventor': np.random.binomial(1, 0.1, num_records),
    'top5cit': np.random.binomial(1, 0.05, num_records)
}

table2a_data = {
    'cz': np.random.choice(commuting_zones, num_records),
    'cohort': np.random.choice(cohorts, num_records),
    'age': np.random.choice(ages, num_records),
    'year': np.random.choice(years, num_records),
    'applicant': np.random.binomial(1, 0.1, num_records),
    'num_grants': np.random.poisson(1, num_records)
}

table1a = pd.DataFrame(table1a_data)
table2a = pd.DataFrame(table2a_data)

# Merge tables based on commuting zone (CZ)
df = pd.merge(table1a, table2a, left_on='par_cz', right_on='cz', how='inner')

# Filter data to include only inventors and those with non-missing CZ information
df = df[(df['applicant'] == 1) & (df['par_cz'].notna()) & (df['cz'].notna())]

# Calculate migration flows
migration_flows = df.groupby(['par_cz', 'cz']).size().reset_index(name='flow')

# Create Sankey diagram of migration flows
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = list(set(migration_flows['par_cz']) | set(migration_flows['cz'])),
      color = "blue"
    ),
    link = dict(
      source = [list(set(migration_flows['par_cz'])).index(i) for i in migration_flows['par_cz']],
      target = [list(set(migration_flows['cz'])).index(i) for i in migration_flows['cz']],
      value = migration_flows['flow']
  ))])

fig.update_layout(title_text="Migration Flows of Inventors", font_size=10)
fig.show()

# Calculate inventor productivity measures by subgroup
df_productivity = df.groupby(['cz', 'cohort', 'age', 'year', 'par_stateabbrv'])[['num_grants', 'top5cit']].mean().reset_index()

# Merge migration flows with productivity measures
df_productivity = pd.merge(df_productivity, migration_flows, left_on='cz', right_on='cz', how='left')

# Create scatter plot of migration rate vs. inventor productivity
fig = px.scatter(df_productivity, x='flow', y='num_grants', color='par_stateabbrv',
                 labels={'flow': 'Migration Rate', 'num_grants': 'Average Patent Grants'},
                 title='Association between Migration Rate and Inventor Productivity')
fig.show()

# Create choropleth map of net migration rate by CZ
df_net_migration = migration_flows.groupby('cz')['flow'].sum().reset_index(name='net_flow')
fig = px.choropleth(df_net_migration, locations='cz', locationmode="USA-states", color='net_flow',
                    scope="usa", color_continuous_scale="RdBu", range_color=(-max(abs(df_net_migration['net_flow'])), max(abs(df_net_migration['net_flow']))),
                    labels={'net_flow': 'Net Migration Rate'})
fig.update_layout(title_text='Net Migration Rate of Inventors by Commuting Zone', geo_scope='usa')
fig.show()

# Create line chart of migration rate and productivity over time
df_time_series = df.groupby(['year', 'cohort', 'age'])[['num_grants', 'top5cit']].mean().reset_index()
fig = px.line(df_time_series, x='year', y='num_grants', color='cohort',
              labels={'year': 'Year', 'num_grants': 'Average Patent Grants'},
              title='Migration Rate and Inventor Productivity over Time')
fig.show()

# Perform statistical tests to compare migration rates and productivity measures across subgroups
subgroups = df['par_stateabbrv'].unique()
for subgroup1, subgroup2 in combinations(subgroups, 2):
    migration_rate1 = df[(df['par_stateabbrv'] == subgroup1)]['flow']
    migration_rate2 = df[(df['par_stateabbrv'] == subgroup2)]['flow']
    t_stat, p_val = ttest_ind(migration_rate1, migration_rate2)
    print(f"t-test results for migration rate between {subgroup1} and {subgroup2}:")
    print(f"t-statistic: {t_stat:.2f}, p-value: {p_val:.4f}")

    productivity1 = df[(df['par_stateabbrv'] == subgroup1)]['num_grants']
    productivity2 = df[(df['par_stateabbrv'] == subgroup2)]['num_grants']
    t_stat, p_val = ttest_ind(productivity1, productivity2)
    print(f"t-test results for inventor productivity between {subgroup1} and {subgroup2}:")
    print(f"t-statistic: {t_stat:.2f}, p-value: {p_val:.4f}")

KeyError: 'cohort'